In [1]:
from scipy.stats import norm
import pandas as pd
import numpy as np
from src.boot import boot_sample, compute_treatment_effects, aggregate_treatment_effects

In [2]:
def algorithm_1_with_se(data, B=1000, alpha=0.05, outcome: str=None, treatment_status: str='treatment', estimand: str='overall', groupby_column: str=None, unit_column: str=None, time_column: str=None, covariates: list=None, treatment_col: str=None, random_seed: int=None):
    """
    Implement Algorithm 1 to compute standard errors and confidence intervals for ATT.
    Also returns the standard errors.
    
    Parameters:
    - data (DataFrame): The dataset containing 'treatment', covariates, and outcomes.
    - B (int): Number of bootstrap samples.
    - alpha (float): Significance level for confidence intervals.
    - treatment_status (str): The column name for the treatment variable.
    - estimand (str): The estimand of interest. Either 'overall', 'cohort', or event.
    - groupby_column (str): The column name for the grouping variable.
    - time_column (str): The column name for the time variable.
    
    Returns:
    - CI (DataFrame): Bootstrapped simultaneous confidence band for ATT.
    - SE (Series): Standard errors for each time period.
    """
    n = len(data)

    # Compute initial ATT and treatment effects to get the number of time periods/parameters
    copy_data = data.copy()
    treatment_effects_og = compute_treatment_effects(data=copy_data, outcome='outcome', treatment=treatment_status, unit_id=unit_column, time_id=time_column, covariates=covariates)
    aggregate_og = treatment_effects_og[treatment_effects_og[treatment_status] == 1]
    ATT = aggregate_treatment_effects(data=aggregate_og, treatment_column=treatment_col, estimand=estimand, groupby_column=groupby_column, time_column=time_column)

    # Convert to NumPy array and get the length
    ATT = np.array(ATT).flatten()
    t_periods = len(ATT)

    # Initialize placeholder for storing bootstrap results
    R_bootstrap = np.zeros((B, t_periods))
    t_test_bootstrap = np.zeros((B, t_periods))

    # Bootstrap sampling and calculation of R(g, t)
    for b in range(B):
        # Step 1: Draw a bootstrap sample
        boot_data = data.copy()
        bootstrap_sample = boot_sample(your_data=boot_data, unit_id=unit_column, time=time_column, random_seed=random_seed)

        # Step pre-2: Compute treatment effects for bootstrap sample
        treatment_effects = compute_treatment_effects(data=bootstrap_sample, outcome='outcome', treatment=treatment_status, unit_id=unit_column, time_id=time_column, covariates=covariates)

        # Filter only treated units
        aggregate = treatment_effects[treatment_effects[treatment_status] == 1]
        
        # Step 2: Compute ATT_star for bootstrap sample
        ATT_star = aggregate_treatment_effects(data=aggregate, treatment_column=treatment_col, estimand=estimand, groupby_column=groupby_column, time_column=time_column)
        ATT_star = np.array(ATT_star).flatten()

        # Compute R(g, t)
        R_gt = np.sqrt(n) * (ATT_star - ATT)
        
        # Store the R(g, t) values
        R_bootstrap[b, :] = R_gt

    # Step 4: Compute bootstrap estimator of the main diagonal of Sigma^(1/2)
    q_75 = np.percentile(R_bootstrap, 75, axis=0)
    q_25 = np.percentile(R_bootstrap, 25, axis=0)
    z_75, z_25 = norm.ppf([0.75, 0.25])
    Sigma_half = (q_75 - q_25) / (z_75 - z_25)

    # Calculate standard errors
    SE = Sigma_half / np.sqrt(n)

    # Step 5: For each bootstrap draw, compute t-test
    for b in range(B):
        t_test_bootstrap[b, :] = np.abs(R_bootstrap[b, :]) / Sigma_half

    # Step 6: Construct confidence band
    c_alpha = np.percentile(t_test_bootstrap, 100 * (1 - alpha), axis=0)
    CI_lower = ATT - c_alpha * Sigma_half / np.sqrt(n)
    CI_upper = ATT + c_alpha * Sigma_half / np.sqrt(n)
    CI = pd.DataFrame({'Lower_Bound': CI_lower, 'Upper_Bound': CI_upper})
    
    return R_bootstrap

In [3]:
# Number of units, time periods, and cohorts
n_units = 10
n_time = 10
n_cohorts = 3

# Random seed for reproducibility
np.random.seed(42)

# Generate a DataFrame
df = pd.DataFrame({
    'unit_id': np.repeat(range(1, n_units + 1), n_time),
    'time_id': list(range(1, n_time + 1)) * n_units,
    'treatment': np.random.choice([0, 1], n_units * n_time),
    'covariate1': np.random.normal(0, 1, n_units * n_time),
    'covariate2': np.random.normal(0, 1, n_units * n_time),
    'cohort': np.random.choice([2010, 2011, 2012], n_units * n_time)
})

# Adding a few never-treated units
df.loc[df['unit_id'].isin([1, 2]), 'treatment'] = 0

# Simulating potential outcomes under the control (Y0)
df['Y0'] = 5 + 0.5 * df['covariate1'] + 0.3 * df['covariate2'] + 0.2 * df['unit_id'] + 0.1 * df['time_id'] + np.random.normal(0, 1, n_units * n_time)

# Defining treatment effect (constant for all units as 2)
df['treatment_effect'] = 2

# Simulating potential outcomes under the treatment (Y1)
df['Y1'] = df['Y0'] + df['treatment_effect']

# Constructing the observed outcome based on treatment status
df['outcome'] = np.where(df['treatment'] == 1, df['Y1'], df['Y0'])

# Adding pre-treatment and post-treatment periods (assuming treatment starts at time 5 for everyone)
df['period_type'] = np.where(df['time_id'] < 5, 'pre-treatment', 'post-treatment')

In [4]:
result_df = compute_treatment_effects(df, 'outcome', 'treatment', 'unit_id', 'time_id', covariates=['covariate1', 'covariate2'])

Skipping column period_type as it appears to be truly textual.


In [5]:
data = result_df.copy()
data = data[data['treatment'] == 1]
data.head()

,unit_id,time_id,treatment,covariate1,covariate2,cohort,Y0,treatment_effect,Y1,outcome,period_type,y_0,t_effects
20,3,1,1,-0.479174,-0.974682,2010,6.312380,2,8.312380,8.312380,pre-treatment,6.312380,2.0
22,3,3,1,-1.106335,1.158596,2010,5.067553,2,7.067553,7.067553,pre-treatment,5.067553,2.0
23,3,4,1,-1.196207,-0.820682,2010,3.813685,2,5.813685,5.813685,pre-treatment,3.813685,2.0
24,3,5,1,0.812526,0.963376,2012,6.469303,2,8.469303,8.469303,post-treatment,6.469303,2.0
25,3,6,1,1.356240,0.412781,2012,10.255939,2,12.255939,12.255939,post-treatment,10.255939,2.0


In [6]:
# Test the aggregate effects function
print(aggregate_treatment_effects(data, 't_effects', estimand='overall'))
print(aggregate_treatment_effects(data, 't_effects', estimand='cohort', groupby_column='cohort', time_column='time_id'))
print(aggregate_treatment_effects(data, 't_effects', estimand='event', groupby_column='cohort' ,time_column='time_id'))

1.9999999999946187
2010    2.0
2011    2.0
2012    2.0
dtype: float64
elapsed_time
-2011    2.0
-2009    2.0
-2008    2.0
-2007    2.0
-2006    2.0
-2005    2.0
-2004    2.0
-2003    2.0
-2002    2.0
-2001    2.0
-2000    2.0
Name: t_effects, dtype: float64


In [7]:
# Number of units, time periods, and cohorts
n_units = 10
n_time = 10
n_cohorts = 3

# Random seed for reproducibility
np.random.seed(42)

# Generate a DataFrame
df = pd.DataFrame({
    'unit_id': np.repeat(range(1, n_units + 1), n_time),
    'time_id': list(range(1, n_time + 1)) * n_units,
    'treatment': np.random.choice([0, 1], n_units * n_time),
    'covariate1': np.random.normal(0, 1, n_units * n_time),
    'covariate2': np.random.normal(0, 1, n_units * n_time),
    'cohort': np.random.choice([2010, 2011, 2012], n_units * n_time)
})

# Adding a few never-treated units
df.loc[df['unit_id'].isin([1, 2]), 'treatment'] = 0

# Simulating potential outcomes under the control (Y0)
df['Y0'] = 5 + 0.5 * df['covariate1'] + 0.3 * df['covariate2'] + 0.2 * df['unit_id'] + 0.1 * df['time_id'] + np.random.normal(0, 1, n_units * n_time)

# Defining treatment effect (constant for all units as 2)
df['treatment_effect'] = 2

# Simulating potential outcomes under the treatment (Y1)
df['Y1'] = df['Y0'] + df['treatment_effect']

# Constructing the observed outcome based on treatment status
df['outcome'] = np.where(df['treatment'] == 1, df['Y1'], df['Y0'])

# Adding pre-treatment and post-treatment periods (assuming treatment starts at time 5 for everyone)
df['period_type'] = np.where(df['time_id'] < 5, 'pre-treatment', 'post-treatment')

In [8]:
df.head()

,unit_id,time_id,treatment,covariate1,covariate2,cohort,Y0,treatment_effect,Y1,outcome,period_type
0,1,1,0,0.738467,0.227460,2011,4.499240,2,6.499240,4.499240,pre-treatment
1,1,2,0,0.171368,1.307143,2012,5.053176,2,7.053176,5.053176,pre-treatment
2,1,3,0,-0.115648,-1.607483,2011,6.911211,2,8.911211,6.911211,pre-treatment
3,1,4,0,-0.301104,0.184634,2010,5.464753,2,7.464753,5.464753,pre-treatment
4,1,5,0,-1.478522,0.259883,2011,5.568139,2,7.568139,5.568139,post-treatment


In [9]:
res = algorithm_1_with_se(df, B=1000, alpha=0.05, treatment_status='treatment', estimand='overall', time_column='time_id', treatment_col='t_effects', outcome='outcome', groupby_column='unit_id', unit_column='unit_id', covariates=['covariate1', 'covariate2'])

Skipping column period_type as it appears to be truly textual.
Skipping column period_type as it appears to be truly textual.
Skipping column period_type as it appears to be truly textual.
Skipping column period_type as it appears to be truly textual.
Skipping column period_type as it appears to be truly textual.
Skipping column period_type as it appears to be truly textual.
Skipping column period_type as it appears to be truly textual.
Skipping column period_type as it appears to be truly textual.
Skipping column period_type as it appears to be truly textual.
Skipping column period_type as it appears to be truly textual.


/workspaces/borusyak_boostrap_se/src/boot.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bs_ID['bs'] = np.arange(N)
/workspaces/borusyak_boostrap_se/src/boot.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bs_ID['bs'] = np.arange(N)
/workspaces/borusyak_boostrap_se/src/boot.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

Skipping column period_type as it appears to be truly textual.
Skipping column period_type as it appears to be truly textual.
Skipping column period_type as it appears to be truly textual.
Skipping column period_type as it appears to be truly textual.
Skipping column period_type as it appears to be truly textual.
Skipping column period_type as it appears to be truly textual.
Skipping column period_type as it appears to be truly textual.
Skipping column period_type as it appears to be truly textual.
Skipping column period_type as it appears to be truly textual.
Skipping column period_type as it appears to be truly textual.


/workspaces/borusyak_boostrap_se/src/boot.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bs_ID['bs'] = np.arange(N)
/workspaces/borusyak_boostrap_se/src/boot.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bs_ID['bs'] = np.arange(N)
/workspaces/borusyak_boostrap_se/src/boot.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

Skipping column period_type as it appears to be truly textual.
Skipping column period_type as it appears to be truly textual.
Skipping column period_type as it appears to be truly textual.
Skipping column period_type as it appears to be truly textual.
Skipping column period_type as it appears to be truly textual.
Skipping column period_type as it appears to be truly textual.
Skipping column period_type as it appears to be truly textual.
Skipping column period_type as it appears to be truly textual.
Skipping column period_type as it appears to be truly textual.
Skipping column period_type as it appears to be truly textual.
Skipping column period_type as it appears to be truly textual.


...

/workspaces/borusyak_boostrap_se/src/boot.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bs_ID['bs'] = np.arange(N)
/workspaces/borusyak_boostrap_se/src/boot.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bs_ID['bs'] = np.arange(N)
/workspaces/borusyak_boostrap_se/src/boot.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

Skipping column period_type as it appears to be truly textual.
Skipping column period_type as it appears to be truly textual.
Skipping column period_type as it appears to be truly textual.
Skipping column period_type as it appears to be truly textual.
Skipping column period_type as it appears to be truly textual.
Skipping column period_type as it appears to be truly textual.
Skipping column period_type as it appears to be truly textual.
Skipping column period_type as it appears to be truly textual.
Skipping column period_type as it appears to be truly textual.
Skipping column period_type as it appears to be truly textual.
Skipping column period_type as it appears to be truly textual.


/workspaces/borusyak_boostrap_se/src/boot.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bs_ID['bs'] = np.arange(N)
/workspaces/borusyak_boostrap_se/src/boot.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bs_ID['bs'] = np.arange(N)
/workspaces/borusyak_boostrap_se/src/boot.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

Skipping column period_type as it appears to be truly textual.
Skipping column period_type as it appears to be truly textual.
Skipping column period_type as it appears to be truly textual.
Skipping column period_type as it appears to be truly textual.
Skipping column period_type as it appears to be truly textual.
Skipping column period_type as it appears to be truly textual.
Skipping column period_type as it appears to be truly textual.
Skipping column period_type as it appears to be truly textual.
Skipping column period_type as it appears to be truly textual.
Skipping column period_type as it appears to be truly textual.
Skipping column period_type as it appears to be truly textual.
Skipping column period_type as it appears to be truly textual.
Skipping column period_type as it appears to be truly textual.


/workspaces/borusyak_boostrap_se/src/boot.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bs_ID['bs'] = np.arange(N)
/workspaces/borusyak_boostrap_se/src/boot.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bs_ID['bs'] = np.arange(N)
/workspaces/borusyak_boostrap_se/src/boot.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

In [10]:
res

array([[ 1.19437793e-11],
       [ 2.65298894e-11],
       [ 2.50688359e-12],
       [ 4.06474854e-11],
       [ 3.32223138e-11],
       [ 2.10187423e-11],
       [ 4.19286827e-11],
       [ 1.92090788e-11],
       [ 1.03850262e-11],
       [ 2.09099404e-11],
       [ 4.03876932e-11],
       [-3.77919918e-11],
       [ 5.48250334e-11],
       [ 3.02469161e-11],
       [-2.92588176e-11],
       [ 1.94089189e-11],
       [ 2.08610906e-11],
       [ 1.11111120e-11],
       [-1.68332015e-11],
       [ 7.11208870e-12],
       [ 2.06346051e-11],
       [ 1.60427227e-11],
       [ 1.78790316e-11],
       [ 1.27409194e-11],
       [ 3.83226784e-11],
       [ 3.22941673e-11],
       [ 1.81299420e-11],
       [ 1.06026299e-11],
       [ 1.69797509e-11],
       [ 5.30286925e-11],
       [-3.11484172e-11],
       [ 2.10520490e-11],
       [ 3.66195962e-11],
       [-2.19158025e-12],
       [ 2.08233430e-11],
       [ 4.62296867e-12],
       [ 3.55893093e-11],
       [-1.24344979e-13],
       [ 1.3

In [11]:
res.describe()

AttributeError: 'numpy.ndarray' object has no attribute 'describe'